In [1]:
# Generate the metadata lookup table using Quine-McCluskey to simplify the information ROM

In [22]:
import pandas as pd
from quine_mccluskey.qm import QuineMcCluskey
from datetime import datetime

In [23]:
data = pd.read_csv("~/Downloads/student_projects_exp.csv")
data = data[~data["Project Name"].isnull()]

In [24]:
len(data)

14

In [25]:
table = {}
maxlen = 0
for _, x in data.iterrows():
    idx = int(x["Project Directory"].split("_")[0].replace("d", ""))
    proj = x["Project Name"]
    stu = str(x["First Name"]) + " " + str(x["Last Name"])
    
    if stu == "nan nan":
        out = proj + " (demo)"
    else:
        out = proj + " by " + stu
        
    out += "\0"
    
    print(idx, x["Project Directory"])
    print(out)
    print()
    
    if maxlen < len(out): maxlen = len(out)
    table[idx] = out
    
table[60] = f"Student projects for Spring 2023 course at Carnegie Mellon\0"
table[61] = f"18-224/18-624/98-154 Intro to Open-Source Chip Design\0"
table[62] = f"Course organized by Anish Singhani and Prof. Bill Nace\0"
table[63] = f"ROM generated {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}\0"

if len(table[60]) > maxlen: maxlen = len(table[60])

1 d01_example_adder
6-Bit Comb. Adder (demo) 

2 d02_example_counter
12-Bit Counter (demo) 

5 d05_meta_info
Tapeout Meta-Info (demo) 

10 d10_jjalacce_connect4
Connect-4 Game by John Alacce 

11 d11_zhexic_i2cdriver
I2C Driver by Zhexi Cao 

12 d12_sjg2_tiny_game_of_life
Tiny Game of Life by Sebastian Garcia 

13 d13_thomaska_cordic
CORDIC by Thomas Kang 

14 d14_siyuanl4_matrixcalc
Matrix Calculator by Siyuan Li 

15 d15_spencer2_pianotiles
Piano Tiles Game by Spencer Li 

16 d16_jaehyun3_bobatc
BobATC by Jaehyun Lim 

17 d17_cporco_clockbox
Clock Box by Christopher Porco 

18 d18_vrajesh_motorcontroller
Motor Controller by Varun Rajesh 

19 d19_gsavant_16bit_serial_cpu
16-bit Serial CPU by Gaurav Savant 

20 d20_zhehuax_16bit_fpu
16-bit FPU by Zhehua Xiang 



In [26]:
assert maxlen < 64
len(table), maxlen

(18, 59)

In [27]:
one_terms = [[] for _ in range(8)]
dc_terms = [[] for _ in range(8)]

for ii in range(8):
    ones = one_terms[ii]
    dc = dc_terms[ii]
    
    for idx in range(64):
        for i in range(64):
            val = (idx << 6) | i

            string = table.get(idx, "\0")
            if i < len(string):
                c = ord(string[i])
                if (c & (1 << ii)) != 0:
                    ones.append(val)
            else:
                dc.append(val)


In [28]:
def convert_minterm(term, bits):
    term = term[::-1]
    s = []
    for i in range(bits):
        if term[i] == "-":
            pass
        elif term[i] == "1":
            s.append(f"index[{i}]")
        elif term[i] == "0":
            s.append(f"~index[{i}]")
        else:
            assert False
    
    if len(s) == 0:
        return "0"
    else:
        return "(" + " & ".join(s) + ")"

In [30]:
statements = ""

for i in range(8):
    print(i)
    qm = QuineMcCluskey()
    s = qm.simplify(ones=one_terms[i], dc=dc_terms[i], num_bits=12)
    mt = " | ".join([convert_minterm(x, 12) for x in s])
    statements += (f"assign val[{i}] = {mt};\n\n")
    

verilog = f"""
`default_nettype none

module my_chip (
    input logic [11:0] io_in,
    input logic clock, reset,
    output logic [11:0] io_out
);

wire [7:0] val;
reg [11:0] index;

always_ff @(posedge clock) begin
    index <= io_in;
    io_out <= {{4'b0000, val}};
end

{statements}

endmodule
"""
    
with open("src/chip.sv", "w+") as f:
    f.write(verilog)

0
1
2
3
4
5
6
7
